# Sensitivity 

In this notebooks, checks to see the impact of various parameters on the Insertion, deletion and Fidelity scores.

In [1]:
!pip install xplique
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import pandas as pd
import os
from google.colab import drive
import json
import itertools

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from spectral_sobol.tf_explainer import WaveletSobol
from xplique.metrics import Deletion, Insertion # evaluation methods
import keras.applications as app # models

In [3]:
# load the labels and the images
source_dir = 'folder/to/imagenet/explique/data'
target_dir = 'path/to/dest'

# number of iterations to compute the scores 
# set to one as it is deterministic wrt to the input data sample
reps = 1

# labels
labels = pd.read_csv(os.path.join(source_dir, 'labels.csv'))
y_wavelet = labels['label'].values
y = np.array([tf.keras.utils.to_categorical(label, 1000) for label in y_wavelet])

# images
images = [cv2.imread(os.path.join(source_dir, img_name))[..., ::-1] for img_name in labels['name'].values]

# vector of samples and metrics
model = app.ResNet50(classifier_activation = "softmax")
preprocessing = app.resnet.preprocess_input

# explain the logits rather than the softmax
model.layers[-1].activation = tf.keras.activations.softmax # 


# preprocess the data
x = preprocessing(np.array(images, copy=True))

# initialize the metrics
deletion = Deletion(model, x, y)
insertion = Insertion(model, x, y)

# helper that will update the file if it exists already
def update(target_dir, filename, data):
  """
  updates the file if it exists, creates and save otherwise
  """

  exists = os.path.exists(os.path.join(target_dir, filename))

  if exists:
    old = json.load(open(os.path.join(target_dir, filename)))
    
    # add the items
    for item in data.keys():
      old[item] = data[item]

    # save the file
    with open(os.path.join(target_dir, filename), 'w') as f:
      json.dump(old, f)

  else:
    with open(os.path.join(target_dir, filename), "w") as f:
      json.dump(data, f)

In [4]:
### Evaluate the effect of the grid size and the number of designs

designs_range = [16]
grid_range = [32]

print(designs_range, grid_range)

scores = {}

for nb_design, grid_size in itertools.product(designs_range, grid_range):

  case = '{}-{}'.format(nb_design, grid_size)
  print(case)

  scores[case] = {
      'insertion' : [],
      'deletion'  : []
  }

  for i in range(reps):
    
    print(i)
    wavelet = WaveletSobol(model, grid_size = grid_size, nb_design = nb_design, batch_size = 512)
    explanations = wavelet(x,y_wavelet)
    
    scores[case]['insertion'].append(insertion(wavelet.spatial_cam))
    scores[case]['deletion'].append(deletion(wavelet.spatial_cam))

    update(target_dir, "parameters.json", scores)

[16] [32]
16-32
0


In [ ]:
### Evaluate the effect of the sampler
from spectral_sobol.sampling import HaltonSequence, LHSampler, MCSampler

# parameters
grid_size = 28
nb_design = 8

# for the chosen grid_size, nb_design parameterization, the Scipy sampler has 
# already been evaluated

samplers = [HaltonSequence(scramble = True), LHSampler(), MCSampler()]
cases = ["halton", "latin", "mc"]

# compute the explanations
scores = {}

for case, sampler in zip(cases, samplers):

  print(case)

  scores[case] = {
      'insertion' : [],
      'deletion'  : []
      }

  for i in range(reps):

    print(i)
    wavelet = WaveletSobol(model, grid_size = grid_size, nb_design = nb_design, sampler = sampler, batch_size = 2048)
    explanations = wavelet(x,y_wavelet)

    scores[case]['insertion'].append(insertion(wavelet.spatial_cam))
    scores[case]['deletion'].append(deletion(wavelet.spatial_cam))

    # at each iteration, save the results (know where to start over if kicked out)
    update(target_dir, 'samplers.json', scores)

halton
0
latin
0
mc
0
